![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [53]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [54]:
cc_apps_nans_replaced = cc_apps.replace("?", np.NaN)

In [55]:
cc_apps_imputed = cc_apps_nans_replaced.copy()

# Imputación de valores faltantes
for col in cc_apps_imputed.columns:
    if cc_apps_imputed[col].dtype == 'object':  # Si la columna es categórica (tipo object)
        # Rellenar con el valor más frecuente
        cc_apps_imputed[col].fillna(cc_apps_imputed[col].value_counts().idxmax(), inplace=True)
    else:
        # Rellenar con la media si es una columna numérica
        cc_apps_imputed[col].fillna(cc_apps_imputed[col].mean(), inplace=True)

In [56]:
# Aplicar codificación one-hot
cc_apps = pd.get_dummies(cc_apps_imputed, drop_first=True)

In [57]:
y = cc_apps.iloc[:, -1].values
X = cc_apps.iloc[:, :-1].values

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [59]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [60]:
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)
y_train_pred = lr.predict(X_train_scaled)
cm = confusion_matrix(y_train_pred, y_train)
print(cm)

[[207   3]
 [  3 270]]


In [61]:
param_grid = {'tol': [0.0001, 0.001] ,'max_iter': [70, 100, 140]}
kf = 5
grd = GridSearchCV(estimator= lr, param_grid= param_grid, cv=kf)
grd.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'max_iter': [70, 100, 140], 'tol': [0.0001, 0.001]})

In [62]:
best_train_score, best_train_params = grd.best_score_, grd.best_params_
print("Best: %f using %s" % (best_train_score, best_train_params))

Best: 0.834300 using {'max_iter': 70, 'tol': 0.0001}


In [63]:
best_model = grd.best_estimator_
best_score = best_model.score(X_test_scaled, y_test)

print("Accuracy of logistic regression classifier: ", best_score)

Accuracy of logistic regression classifier:  0.8019323671497585
